In [1]:
# %pip install langchain gradio chromadb sentence-transformers unstructured markdown

In [ ]:
import torch

In [4]:
from langchain_community.vectorstores import Chroma
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embedder = HuggingFaceEmbeddings(model_name="../autodl-tmp/model/bge-m3")
embedder.client.requires_grad_(False)
    
vectordb = Chroma(
    persist_directory="./vectorDB100/QA",
    embedding_function = embedder
)


{'ids': ['00062700-f740-4a46-b775-65b4e612bb32',
  '00066429-39ff-46e2-931c-9d424d53e77f',
  '00073f0a-4a03-45da-8d8b-8a032d9ec16d',
  '00075bf6-285b-474a-a7e9-a3c39354b779',
  '0008c6c2-f202-4f3d-95f7-d2251b95a247',
  '0008fe1f-a383-4a1d-8d6a-66c869085004',
  '000929b0-1ea3-42da-bf1c-9f0e2232ab6f',
  '000abaee-6bce-454f-9ee6-9db85e2a0b1c',
  '000ac2fd-d22f-48b9-a5b4-e7d48cdff1d2',
  '000b7b80-043a-4949-86c4-105fd0f177d6',
  '000c393e-039e-42fc-a32f-dcf35d968ea1',
  '000e7b52-50d7-46bb-8f16-85a3ddbc2a92',
  '000ed979-f61b-4409-8998-ac2d49b36cb5',
  '000f48ad-a253-442e-9ae4-e2dd023d4142',
  '00130907-bbed-4c81-b505-49c641aae218',
  '001329e6-9cff-461a-89de-4fb61e80b4af',
  '0015048b-98ec-47b9-91db-64cbcf1ccfa2',
  '00185ab0-e466-46ad-a031-9426d6d0d9b1',
  '001cd413-943d-4fa1-b850-ad356898f09a',
  '0026c866-5d52-41ed-9305-4398bc36fa78',
  '00298042-a538-42d6-a164-b5bde7ab59f6',
  '002fd31b-1060-4013-92cb-e196551f8eff',
  '003011a6-ca24-4b10-a165-6acb72a38070',
  '003521d8-7663-44dc-867d-

In [5]:
info = vectordb.get()


In [9]:
len(info["ids"])

27521

## QA data preparation

In [7]:
import pandas as pd
import numpy as np

df = pd.read_csv("./raw/medical_QA.csv", encoding="utf-8-sig")

In [3]:
df

,column2,department,title,ask,answer
0,_，血糖,"""内分泌科""","""糖尿病还会进行遗传吗？""","""糖尿病有隔代遗传吗？我妈是糖尿病，很多年了，也没养好，我现在也是，我妹子也是，我儿子现在二...","""2型糖尿病的隔代遗传概率为父母患糖尿病，临产的发生率为40%，比一般人患糖尿病，疾病，如何..."
1,_胰岛素,"""内分泌科""","""糖尿病患者能够进行缓解吗？""","""可是年纪大了吧，以前累积下来的大大小小的病都在这一次爆发了。我得了糖尿病。看过医生，也有在...","""糖尿病的治疗主要有饮食经营，活动，口服降糖药物治疗和麻醉胰岛素治疗。药物治疗，均需要有互相..."
2,_奥美拉唑_达喜片,"""消化科""","""哪家医院能治胃反流""","""烧心，打隔，咳嗽低烧，以有4年多""","""建议你用奥美拉唑同时，加用吗丁啉或莫沙必利或援生力维，另外还可以加用达喜片"""
3,_卡马西平,"""神经科""","""想知道癫痫长期用药的危害有什么？""","""查出来癫痫十几年一直在喝药压但是药三分长期喝药一定是不好癫痫长期用药的危害有什么？""","""根据你叙述的情有癫目前没正式救感觉症状越来越严重。在这种情况建议在医生的指导下治疗癫痫症状..."
4,_卡托普利,"""心血管科""","""高血压出现头晕，脸红的症状怎么办？""","""打搅了医生，情况是这样子的，我爷爷现在是个高血压患者，可是他最近说他经常头晕，很痛，然后我...","""高血压再次出现头晕脸红的症状，应当还是由血管扩展引发的，这种情况再次出现头晕一般跟脑血管痉..."
...,...,...,...,...,...
47853,_青霉胺_~_二巯丙醇,"""肾内科""","""吃什么有助于肾功能的恢复""","""中毒至今有18天，出现水肿，呼吸困难，腿抽筋""","""急性砷中毒解毒药选用①二琉丙磺钠；②二巯丙醇；③青霉胺。发生肾功能衰竭应及早用血液透析，并..."
47854,_甲硝唑_香砂养胃丸参苓白术散,"""消化科""","""胆汁反流性胃炎伴糜烂怎样治疗和饮食那些比较适""","""2011.9月至今胆汁反流性胃炎伴糜烂，腹胀，想吐，嗳气，打嗝，右上腹胀痛""","""你好根据你的情况属于胃炎引起的，慢性胃炎引起的症状，是指由不同病因所致的胃粘膜慢性炎症。其..."
47855,_、奥米拉_如甲硝唑_、阿莫西林胶,"""消化科""","""四十三岁慢性胃炎胃酸多怎么治疗""","""2008年消化不良反酸吃过饭胃翻淮安第2人民医院""","""你好，一般胃炎多是 由于幽门螺杆菌感染引起的，可以采用三联疗法治疗，如甲硝唑片、奥米拉唑、..."
47856,_克霉唑_伊曲康唑胶囊,"""普通内科""","""三岁小男孩虫虫痛""","""一个月前就这样了，虫虫前面是青的，很痛。""","""你好，根据你的描述，可能是霉菌性包皮龟头炎，属白色念珠菌感染引起.建议服用伊曲康唑胶囊，外..."


In [16]:
with open("./docs/demo/QA_Reference_mini.txt", "w", encoding="utf-8") as outfile:
    np.random.seed(118010142)
    randidx = np.random.randint(df.shape[0], size=1000)
    for idx in randidx:
        row = df.iloc[idx]
        row_content = "标题： {} 问题：{} 医生回答：{}。\n\n".format(row["title"], row["ask"], row["answer"])
        outfile.write(row_content)

In [17]:
file_name = "./docs/demo/QA_Reference_"
cnt = 0
num = 0
content = []
for idx, row in df.iterrows():
    row_content = "标题： {} 问题：{} 医生回答：{}。\n\n".format(row["title"], row["ask"], row["answer"])
    outfile.write(row_content)
    cnt += 1
    content.append(row_content)
    if cnt == 1000:
        with open(os.path.join(file_name, f"{num}.txt"), "w") as outfile:
            outfile.writelines(content)
        cnt = 0
        num += 1

## Files loading. Using all txt and md files.

In [2]:
# import os 
# def get_files(dir_path):
#     # args：dir_path，目标文件夹路径
#     file_list = []
#     for filepath, dirnames, filenames in os.walk(dir_path):
#         # os.walk 函数将递归遍历指定文件夹
#         for filename in filenames:
#             # 通过后缀名判断文件类型是否满足要求
#             if filename.endswith(".md"):
#                 # 如果满足要求，将其绝对路径加入到结果列表
#                 file_list.append(os.path.join(filepath, filename))
#             elif filename.endswith(".txt"):
#                 file_list.append(os.path.join(filepath, filename))
#     return file_list

In [20]:
from tqdm import tqdm
from langchain.document_loaders import UnstructuredFileLoader
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain_community.document_loaders.csv_loader import UnstructuredCSVLoader
import os

# def get_text(dir_path):
#     # args：dir_path，目标文件夹路径
#     # 首先调用上文定义的函数得到目标文件路径列表
#     file_lst = get_files(dir_path)
#     # docs 存放加载之后的纯文本对象
#     docs = []
#     # 遍历所有目标文件
#     for one_file in tqdm(file_lst):
#         file_type = one_file.split('.')[-1]
#         if file_type == 'md':
#             loader = UnstructuredMarkdownLoader(one_file)
#         # elif file_type == 'txt':
#         #     loader = UnstructuredFileLoader(one_file)
#         # elif file_type == 'csv':
#         #     loader = UnstructuredCSVLoader(one_file)
#         else:
#             # 如果是不符合条件的文件，直接跳过
#             continue
#         docs.extend(loader.load())
#     return doc

def get_files(dir_path, types=[".md"]):
    """
    Args:
        dir_path (str): Directory for RAG docs
        types (list, optional): File types of RAG docs. Only support markdown and csv. Defaults to [".md"].

    Returns:
        _type_: list of file paths.
    """
    file_list = []
    for filepath, dirnames, filenames in os.walk(dir_path):
        for filename in filenames:
            for file_type in types:
                if filename.endswith(file_type):
                    file_list.append(os.path.join(filepath, filename))
                    continue

    return file_list


def get_text(dir_path, types=[".md"]):
    """
    Args:
        dir_path (str): Directory for RAG docs
        types (list, optional): File types of RAG docs. Only support markdown and csv. Defaults to [".md"].

    Returns:
        _type_: list of docs.
    """
    file_list = get_files(dir_path, types)
    docs = []
    for one_file in tqdm(file_list):
        file_type = one_file.split('.')[-1]
        if file_type == 'md':
            loader = UnstructuredMarkdownLoader(one_file)
        elif file_type == 'txt':
            loader = UnstructuredFileLoader(one_file)
        else:
            continue
        docs.extend(loader.load())
    return docs


## Get embeddings

In [29]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

docs = get_text("./demo_docs", types=["txt"])

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=150)
split_docs = text_splitter.split_documents(docs)

100%|██████████| 1/1 [00:03<00:00,  3.36s/it]


In [30]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="../autodl-tmp/model/bge-m3")

/root/miniconda3/envs/nlp/lib/python3.10/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/root/miniconda3/envs/nlp/lib/python3.10/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [61]:
from langchain.vectorstores import Chroma

# 定义持久化路径
persist_directory = 'vectorDB/chroma'
try:
    vectordb = Chroma(
    persist_directory=persist_directory, 
    # embedding_function=embeddings
    )
    vectordb.delete_collection()
    print("Successfully initialize the ChromaDB.")
except:
    raise Exception("Reset Error")
vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=embeddings,
    persist_directory=persist_directory 
)
vectordb.persist()

Successfully initialize the ChromaDB.


In [63]:
persist_directory = 'vectorDB/chroma'

# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory, 
    embedding_function=embeddings
)
len(vectordb.get()["ids"])

626

In [9]:
len(split_docs)

4157

In [10]:
len(vectordb.get()["ids"])

16661

## Language Model

ChatGLM

In [11]:
from langchain.llms.base import LLM
from typing import Any, List, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from transformers import AutoTokenizer, AutoModelForCausalLM

class ChatGLM_LLM(LLM):
    # 基于本地 InternLM 自定义 LLM 类
    tokenizer : AutoTokenizer = None
    model: AutoModelForCausalLM = None

    def __init__(self, model_path :str):
        # model_path: InternLM 模型路径
        # 从本地初始化模型
        super().__init__()
        print("正在从本地加载模型...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True).to(torch.bfloat16).cuda()
        self.model = self.model.eval()
        print("完成本地模型的加载")

    def _call(self, prompt : str, stop: Optional[List[str]] = None,
                run_manager: Optional[CallbackManagerForLLMRun] = None,
                **kwargs: Any):
        # 重写调用函数
        response, history = self.model.chat(self.tokenizer, prompt , history=[])
        return response
        
    @property
    def _llm_type(self) -> str:
        return "ChatGLM3-6B"

## Get vectorDB

In [25]:
from langchain.vectorstores import Chroma
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import os

# 定义 Embeddings
# embeddings = HuggingFaceEmbeddings(model_name="autodl-tmp/sentence-transformer")

# 向量数据库持久化路径
persist_directory = 'data_base/vector_db/chroma'

# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory, 
    # embedding_function=embeddings
)
len(vectordb.get()["ids"])

In [13]:
# from LLM import ChatGLM_LLM
llm = ChatGLM_LLM(model_path = "../autodl-tmp/model/chatglm3-6b")

Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


正在从本地加载模型...


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

完成本地模型的加载


In [14]:
llm.predict("你是谁？")

/root/miniconda3/envs/nlp/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'我是一个名为 ChatGLM3-6B 的人工智能助手，是基于清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。'

## Template

In [15]:
from langchain.prompts import PromptTemplate

# 我们所构造的 Prompt 模板
template = """请使用以下参考内容来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
有用的回答:"""

# 调用 LangChain 的方法来实例化一个 Template 对象，该对象包含了 context 和 question 两个变量，在实际调用时，这两个变量会被检索到的文档片段和用户提问填充
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],template=template)

In [16]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm,retriever=vectordb.as_retriever(),return_source_documents=True,chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

## QA

Comparison

In [17]:
question = "你是谁？"
result = qa_chain({"query": question})
print("检索问答链回答 question 的结果：")
print(result["result"])

/root/miniconda3/envs/nlp/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


检索问答链回答 question 的结果：
我是皇帝身边的人，我的名字是甄嬛。


In [18]:
# 检索问答链回答效果
question = "什么是 Self LLM"
result = qa_chain({"query": question})
print("检索问答链回答 question 的结果：")
print(result["result"])

# 仅 LLM 回答效果
result_2 = llm(question)
print("大模型回答 question 的结果：")
print(result_2)

检索问答链回答 question 的结果：
Self LLM是指通过自身数据进行微调的现有 LLM。通过在自身数据上进行训练和调整，可以构建出一个更适合自身任务和数据的模型。


/root/miniconda3/envs/nlp/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


大模型回答 question 的结果：
Self-supervised learning (SSL) is a subfield of machine learning that focuses on training models on unlabeled data. Unlike supervised learning, where the data is labeled with the correct output for each example, SSL aims to learn useful features or representations from the unlabeled data itself.

One common approach in SSL is to use a pre-trained model as a starting point and fine-tune it on the unlabeled data. This is known as transfer learning. Another approach is to train a model from scratch on the unlabeled data, without any prior knowledge or pre-trained model.

Self-supervised learning has been applied to a wide range of tasks, such as image generation, video generation, language modeling, and recommendation systems. It has shown promising results in many cases, particularly when the data is abundant and labeled is expensive or difficult to obtain. However, SSL is still an active area of research and there is much room for improvement in terms of understandin